# Pipelines to evaluate fine-mapping benchmarks

This document contains several workflows to extract results from fine-mapping benchmarks and make plots for (mostly) SuSiE manuscript.

In [ ]:
[global]
parameter: cwd = path('~/GIT/lab-dsc/dsc-finemap')
parameter: version = '20190610'
# set to 0 to use estimated prior, or another number to use specified prior
# as configured in the DSC
parameter: susie_prior = [0.1, 0.0]
# null weight options to evaluate
parameter: null_weight = [0.0]
parameter: output_dir = path(f'{cwd:a}/output/susie_paper')

def fmtP(x):
    return str(x).replace(".", "p")

## Query results

In [ ]:
[*_1]
output: f'{output_dir}/susie_paper/result_{version}.rds'
R: expand = '${ }', workdir = cwd
    dap_out = dscrutils::dscquery(${dirname:br}, 
                        target = "liter_data.dataset liter_data lm_less lm_less.pve lm_less.n_signal fit_dap")
    susie_out = dscrutils::dscquery(${dirname:br}, 
                        target = "liter_data.dataset liter_data lm_less lm_less.pve lm_less.n_signal fit_susie.prior_var fit_susie.null_weight fit_susie")
    caviar_out = dscrutils::dscquery(${dirname:br}, 
                        target = "liter_data.dataset liter_data lm_less lm_less.pve lm_less.n_signal fit_caviar.args fit_caviar")
    finemap_out = dscrutils::dscquery(${dirname:br}, 
                        target = "liter_data.dataset liter_data lm_less lm_less.pve lm_less.n_signal fit_finemap.args fit_finemap")
    saveRDS(list(dap=dap_out, susie=susie_out, caviar=caviar_out, finemap=finemap_out), ${_output:r})

## PIP calibration